In [2]:
import os
import pandas as pd
import numpy as np

import sys
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

from dotenv import load_dotenv

load_dotenv()

True

In [3]:
client_credentials_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

Blink-182 song

In [4]:
# Songs
carousel = "3gVXAW2pduBl7tzjAGUVRb"
carousel_aa = sp.audio_analysis(carousel)

In [5]:
carousel_aa.keys()

dict_keys(['meta', 'track', 'bars', 'beats', 'sections', 'segments', 'tatums'])

In [6]:
len(carousel_aa["segments"])

775

In [68]:
process_pitch(carousel_aa["segments"])

0


divido a movimentação de um ponto em 12 direções

In [38]:
sp.album_tracks("0GqpoHJREPp0iuXK3HzrHk")["items"][0]['id']

'2e61kmae4HhbuS9hYONQ0A'

In [39]:
len(sp.album_tracks("0GqpoHJREPp0iuXK3HzrHk")["items"])

8

In [13]:
ids_houses_of_holy = [sp.album_tracks("0GqpoHJREPp0iuXK3HzrHk")["items"][i]['id'] for i in range(len(sp.album_tracks("0GqpoHJREPp0iuXK3HzrHk")["items"]) - 1)]

In [19]:
-carousel_aa["segments"][0]["loudness_max"]

60.0

In [23]:

def getTrackIDs(user, playlist_id):
    ids = []
    playlist = sp.user_playlist(user, playlist_id)
    for item in playlist['tracks']['items']:
        track = item['track']
        ids.append(track['id'])
    return ids

def get_song_id_metrics(song_id):
    song_audio = sp.audio_analysis(song_id)
    coords = []
    # print(len(song_audio["segments"]))
    for i in range(len(song_audio["segments"])-1):
        data = process_pitch(song_audio["segments"][i]['pitches'])
        data.extend([-song_audio["segments"][i]["loudness_max"]])
        coords.append(data)
    return coords

def process_pitch(pitch_list):
    # notas = ["C","C#","D","D#","E","F","F#","G","G#","A","A#","B"]
    # print(len(pitch_list))
    # try:
    min_max_list = normalize(pitch_list)
    coordinates = get_coordinates(min_max_list)
    # coordinates = coordinates[9]
    coordinates = list(np.sum(coordinates, axis=0))
        # print(coordinates)
        # pitch_dict = [coordinates[i] for i in range(len(notas)-1)]
    # except:
        # print("erro")
        # return
    # return pitch_dict
    return coordinates

def normalize(input_list):
    min_val = min(input_list)
    max_val = max(input_list)
    if max_val == min_val:
        return [0 for x in input_list]
    normalized_list = [(x - min_val) / (max_val - min_val) for x in input_list]
    return normalized_list

def get_coordinates(normalized_input):
    i = 0
    coordinates = []
    for i in range(len(normalized_input) - 1):
        # print(normalized_input[i])
        cos = round(np.cos(i * 2 * np.pi/12), 5) * normalized_input[0]
        sin = round(np.sin(i * 2 * np.pi/12), 5) * normalized_input[1]
        # print((cos, sin))
        coordinates.append((cos, sin))
    return coordinates
        
        

In [25]:
database = {}
for i,song_id in enumerate(ids_houses_of_holy):
    database[f"{i}"] = get_song_id_metrics(song_id)

In [26]:
database['1']

[[-0.13992624454148467, 0.1533842794759826, 34.744],
 [-0.04647426212590299, 0.041279669762641885, 30.964],
 [0.0, 0.0010172939979654113, 29.282],
 [-0.001735531062124248, 0.0005010020040080161, 28.723],
 [0.0, 0.003015075376884422, 33.118],
 [-0.0008686359077231697, 0.0010030090270812442, 40.711],
 [-0.0008695080321285139, 0.0020080321285140573, 38.303],
 [-0.00174251509054326, 0.0, 42.497],
 [-0.0026243333333333336, 0.0015151515151515145, 36.981],
 [-0.02557522403258656, 0.01782077393075355, 41.535],
 [-0.04759432112068965, 0.10129310344827588, 24.903],
 [0.0, 0.008048289738430588, 22.383],
 [0.0, 0.004012036108324977, 28.796],
 [-0.0008686359077231697, 0.0055165496489468415, 38.426],
 [-0.0026085240963855424, 0.0035140562248996005, 32.257],
 [0.0, 0.007522567703109327, 33.994],
 [0.0, 0.01407035175879396, 28.604],
 [0.0, 0.02896341463414634, 32.134],
 [-0.007809889779559117, 0.0030060120240480966, 24.879],
 [-0.0008928144329896913, 0.0, 18.858],
 [-0.029014203045685274, 0.0, 22.919]

In [23]:
import json

with open("houses_of_the_holy.json", "w") as fp:
    json.dump(database , fp, indent = 4)